In [2]:
# Nutrition data collected from: 
# https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/food-surveys-research-group/docs/fndds-download-databases/

# Cost data collected from:
# https://www.fns.usda.gov/resource/cnpp-data

In [3]:
import pandas as pd
import numpy as np

import cvxpy as cp
import math

## LP Example

In [7]:
fake_food_names = ['Milk','Bread','Cheese','Chicken','Beans']

In [4]:
# n is number of variabless we are solving for
n = 5

# a is cost
a = np.array([1.0,2.0,3.0,4.0,5.0]).reshape(5,1)

food1 = [5,0,15]
food2 = [11,4,33]
food3 = [11,1,15]
food4 = [4,32,0]
food5 = [0,0,30]

# b is nutrition
b = np.array([food1,food2,food3,food4,food5]).reshape(5,3)

# d is daily nutrition contraints
d = np.array([70,50,300])

# g is the max nutrition allowed 
g = np.array([90,math.inf,500])

In [5]:
x = cp.Variable(n)

objective = cp.Minimize(cp.sum(a.T*x))

contraints = [0 <= x, b.T*x >= d, b.T*x <= g]

prob = cp.Problem(objective, contraints)

result = prob.solve()

In [6]:
print("Daily Price of Diet: $",round(result,2))

Daily Price of Diet: $ 24.35


In [15]:
print('Amount per Food:')

for i in range(len(fake_food_names)):
    amt_food = round(x.value[i],2)
    if amt_food != 0:
        print(fake_food_names[i],':',amt_food)

Amount per Food:
Bread : 7.98
Chicken : 0.57
Beans : 1.23


## Load & Merge Data

In [69]:
nutrition_pd = pd.read_excel('food_nutrition_2016',header=1)

In [70]:
# Edit Nutrition Set
nutrition_pd.drop(['WWEIA Category code','WWEIA Category description'],inplace=True,axis=1)

In [71]:
nutrition_pd.head()

,Food code,Main food description,Energy (kcal),Protein (g),Carbohydrate (g),"Sugars, total (g)","Fiber, total dietary (g)",Total Fat (g),"Fatty acids, total saturated (g)","Fatty acids, total monounsaturated (g)",...,20:1 (g),22:1 (g),18:2 (g),18:3 (g),18:4 (g),20:4 (g),20:5 n-3 (g),22:5 n-3 (g),22:6 n-3 (g),Water (g)
0,11000000,"Milk, human",70,1.03,6.89,6.89,0.0,4.38,2.009,1.658,...,0.040,0.0,0.374,0.052,0.0,0.026,0.0,0.0,0.0,87.50
1,11100000,"Milk, NFS",51,3.27,4.85,5.08,0.0,2.04,1.232,0.541,...,0.001,0.0,0.070,0.030,0.0,0.000,0.0,0.0,0.0,89.13
2,11111000,"Milk, whole",61,3.15,4.80,5.05,0.0,3.25,1.865,0.812,...,0.000,0.0,0.120,0.075,0.0,0.000,0.0,0.0,0.0,88.13
3,11111100,"Milk, low sodium, whole",61,3.10,4.46,4.46,0.0,3.46,2.154,0.999,...,0.000,0.0,0.078,0.050,0.0,0.000,0.0,0.0,0.0,88.20
4,11111150,"Milk, calcium fortified, whole",61,3.15,4.80,5.05,0.0,3.25,1.865,0.812,...,0.000,0.0,0.120,0.075,0.0,0.000,0.0,0.0,0.0,88.13


In [72]:
print(nutrition_pd.shape[0],'foods')

8690 foods


In [73]:
price_pd = pd.read_excel('food_prices_2004',header=1)

In [74]:
code = []
food =[]
for biggy in price_pd.index:
    code.append(biggy[0])
    food.append(biggy[1])

In [75]:
price_pd['Food code'] = code
price_pd['Main food description'] = food
price_pd.index = range(price_pd.shape[0])
price_pd.head()

,$/100 g,Food code,Main food description
0,0.000000,11000000,"Milk, human"
1,0.110243,11100000,"Milk, NFS"
2,0.146166,11111000,"Milk, cow's, fluid, whole"
3,0.170330,11111150,"Milk, calcium fortified, cow's, fluid, whole"
4,0.089162,11112000,"Milk, cow's, fluid, other than whole, NS as to..."


In [82]:
print(price_pd.shape[0],'foods')

4634 foods


In [79]:
food_data_pd = pd.merge(price_pd, nutrition_pd, how='inner', on=['Food code','Main food description'])

In [80]:
food_data_pd.head()

,$/100 g,Food code,Main food description,Energy (kcal),Protein (g),Carbohydrate (g),"Sugars, total (g)","Fiber, total dietary (g)",Total Fat (g),"Fatty acids, total saturated (g)",...,20:1 (g),22:1 (g),18:2 (g),18:3 (g),18:4 (g),20:4 (g),20:5 n-3 (g),22:5 n-3 (g),22:6 n-3 (g),Water (g)
0,0.000000,11000000,"Milk, human",70,1.03,6.89,6.89,0.0,4.38,2.009,...,0.040,0.0,0.374,0.052,0.0,0.026,0.0,0.0,0.0,87.50
1,0.110243,11100000,"Milk, NFS",51,3.27,4.85,5.08,0.0,2.04,1.232,...,0.001,0.0,0.070,0.030,0.0,0.000,0.0,0.0,0.0,89.13
2,0.018976,11121100,"Milk, dry, reconstituted, whole",76,4.01,5.86,5.86,0.0,4.08,2.554,...,0.000,0.0,0.070,0.031,0.0,0.000,0.0,0.0,0.0,85.04
3,0.303963,11410000,"Yogurt, NS as to type of milk or flavor",63,5.25,7.04,7.04,0.0,1.55,1.000,...,0.000,0.0,0.031,0.013,0.0,0.000,0.0,0.0,0.0,85.06
4,0.277827,11459990,"Yogurt, frozen, NS as to flavor, NS as to type...",139,8.00,21.00,21.00,0.0,2.50,1.509,...,0.000,0.0,0.047,0.023,0.0,0.000,0.0,0.0,0.0,68.08


In [83]:
print(food_data_pd.shape[0],'foods')

2225 foods


In [86]:
for i in food_data_pd['Main food description'].values:
    print(i)

Milk, human
Milk, NFS
Milk, dry, reconstituted, whole
Yogurt, NS as to type of milk or flavor
Yogurt, frozen, NS as to flavor, NS as to type of milk
Yogurt, frozen, flavors other than chocolate, NS as to type of milk
Yogurt, frozen, chocolate, NS as to type of milk
Yogurt, frozen, NS as to flavor, lowfat milk
Yogurt, frozen, chocolate, lowfat milk
Yogurt, frozen, flavors other than chocolate, lowfat milk
Yogurt, frozen, chocolate, nonfat milk
Yogurt, frozen, flavors other than chocolate, with sorbet or sorbet-coated
Yogurt, frozen, flavors other than chocolate, nonfat milk
Yogurt, frozen, chocolate, nonfat milk, with low-calorie sweetener
Yogurt, frozen, flavors other than chocolate, nonfat milk, with low-calorie sweetener
Yogurt, frozen, flavors other than chocolate, whole milk
Yogurt, frozen, cone, flavors other than chocolate
Yogurt, frozen, cone, flavors other than chocolate, lowfat milk
Yogurt, frozen, cone, chocolate, lowfat milk
Whey, sweet, dry
Cream, NS as to light, heavy, or 

Corn, white, cooked, NS as to form, NS as to fat added in cooking
Corn, white, cooked, from fresh, fat not added in cooking
Corn, white, cooked, from frozen, fat not added in cooking
Corn, white, cooked, from canned, fat not added in cooking
Corn, yellow, canned, low sodium, fat not added in cooking
Cucumber, cooked, fat not added in cooking
Eggplant, cooked, NS as to fat added in cooking
Eggplant, cooked, fat not added in cooking
Garlic, cooked
Hominy, cooked, fat not added in cooking
Hominy, cooked, fat added in cooking
Mushrooms, cooked, NS as to form, NS as to fat added in cooking
Mushrooms, cooked, from fresh, NS as to fat added in cooking
Mushrooms, cooked, NS as to form, fat not added in cooking
Mushrooms, cooked, from fresh, fat not added in cooking
Mushrooms, cooked, from frozen, fat not added in cooking
Mushrooms, cooked, from canned, fat not added in cooking
Okra, cooked, NS as to form, NS as to fat added in cooking
Okra, cooked, from fresh, NS as to fat added in cooking
Okr

In [ ]:
# See if they have all the necessary fruits and veggies 

In [87]:
veggies = pd.read_excel('veggie_list',header=1)

In [97]:
# veggies['Unnamed: 1','Unnamed: 4','Unnamed: 6']
v1 = veggies['Unnamed: 1'].dropna().values
v2 = veggies['Unnamed: 4'].dropna().values
v3 = veggies['Unnamed: 6'].dropna().values

In [111]:
names = [*v1,*v2, *v3]
food_names = []
for veggie in names:
    if ('Other' in veggie) or (',' in veggie) or ('(' in veggie):
        continue
    elif '*' in veggie:
        veggie = veggie[:-1]
    food_names.append(veggie)

In [112]:
food_names

['Artichoke',
 'Arugula',
 'Asparagus',
 'Baby Bok Choy',
 'Beets',
 'Belgian Endive',
 'Bell Peppers',
 'Bitter Melons',
 'Black Salsify',
 'Bok Choy',
 'Broccoflower',
 'Broccoli',
 'Broccoli Rabe',
 'Broccolini',
 'Brussels Sprouts',
 'Carrot',
 'Cauliflower',
 'Celeriac',
 'Celery ',
 'Chinese Long Beans',
 'Collard Greens',
 'Corn',
 'Cucumber',
 'English Cucumber',
 'Dandelion Greens',
 'Eggplant',
 'Endive',
 'Escarole',
 'Fava Beans',
 'Fennel',
 'Fiddlehead Ferns',
 'Apple',
 'Apricots',
 'Avocado',
 'Banana',
 'Black Currants',
 'Blackberries',
 'Blueberries',
 'Boysenberries',
 'Breadfruit',
 'Cactus',
 'Cantaloupe',
 'Cape Gooseberries',
 'Cherimoya',
 'Cherries',
 'Clementines',
 'Coconut',
 'Cranberries',
 'Dates',
 'Durian*',
 'Elderberries',
 'Feijoa',
 'Fig',
 'Grapefruit',
 'Grapes',
 'Guava',
 'Huckleberries',
 'Jackfruit',
 'Kiwifruit',
 'Kumquat',
 'Lemon',
 'Lime',
 'Loganberries',
 'Lychee',
 'Mango',
 'Galangal Root',
 'Garlic',
 'Ginger Root',
 'Green Beans',
 

In [105]:
included_foods = food_data_pd['Main food description'].values

In [116]:
include_count = 0
include_foods = []
print("Foods not included:")
for food in food_names:
    is_in = False
    for food2 in included_foods:
        if food in food2:
            is_in = True
            include_count += 1
            include_foods.append(food)
            break
    if not is_in:
        print(food)
print('\n',include_count,'out of',len(food_names),'foods included (',round(100*include_count/len(food_names),2),'%)')

Foods not included:
Arugula
Baby Bok Choy
Belgian Endive
Bell Peppers
Bitter Melons
Black Salsify
Bok Choy
Broccoflower
Broccoli Rabe
Broccolini
Brussels Sprouts
Chinese Long Beans
Collard Greens
English Cucumber
Dandelion Greens
Endive
Fava Beans
Fennel
Fiddlehead Ferns
Black Currants
Breadfruit
Cape Gooseberries
Cherimoya
Clementines
Dates
Durian*
Elderberries
Feijoa
Huckleberries
Jackfruit
Kiwifruit
Kumquat
Loganberries
Galangal Root
Ginger Root
Green Beans
Leaf Lettuce
Leek
Mustard Greens
Mulberries
Passion Fruit
Plaintain
Pluot
Pummelo
Quince
Red Currants
Sapote
Star Fruit
Ugli Fruit
Wild Bluberries
Radish 
Snow Pea Vines
Snow Peas
Sugar Snap Peas
Sweet Potato
Taro Leaf
Tomatillo
Wasabi Root
Winter Melon
Yam
Yucca Root

 65 out of 126 foods included ( 51.59 %)


In [115]:
include_foods

['Artichoke',
 'Asparagus',
 'Beets',
 'Broccoli',
 'Carrot',
 'Cauliflower',
 'Celeriac',
 'Celery ',
 'Corn',
 'Cucumber',
 'Eggplant',
 'Escarole',
 'Apple',
 'Apricots',
 'Avocado',
 'Banana',
 'Blackberries',
 'Blueberries',
 'Boysenberries',
 'Cactus',
 'Cantaloupe',
 'Cherries',
 'Coconut',
 'Cranberries',
 'Fig',
 'Grapefruit',
 'Grapes',
 'Guava',
 'Lemon',
 'Lime',
 'Lychee',
 'Mango',
 'Garlic',
 'Jicama',
 'Kale',
 'Kohlrabi',
 'Mushrooms',
 'Okra',
 'Olives',
 'Onion',
 'Parsnips',
 'Peas',
 'Potato',
 'Nectarine',
 'Orange',
 'Papaya',
 'Peach',
 'Pear',
 'Persimmon',
 'Pineapple',
 'Plum',
 'Pomegranate',
 'Raspberries',
 'Strawberries',
 'Tangerine',
 'Watermelon',
 'Pumpkin',
 'Radicchio',
 'Rhubarb',
 'Rutabaga',
 'Spinach',
 'Tomato',
 'Turnip',
 'Watercress',
 'Zucchini']